In [73]:
from model.binomial import InterestRate, EuropeanVanillaModel, AmericanModel
import pandas as pd

#for filling column headers
columns = lambda x: ['t = ' + str(i) for i in range(x + 1)]

<h3>Vanilla European Call Example</h3>

In [74]:
r = InterestRate(0.12, short_term=False)
european_call = EuropeanVanillaModel(20, 21, .5, 1.1, .9, r, 2, option='call')

<h5>Tree of stock prices</h5>

In [75]:
s_tree = european_call.stock_tree()
#s_tree.T will transpose the tree so it looks like how we build them by hand
st_df = pd.DataFrame(s_tree.T, columns=columns(european_call.n))
st_df

,t = 0,t = 1,t = 2
0,20.0,22.0,24.2
1,0.0,18.0,19.8
2,0.0,0.0,16.2


<h5>Tree of derivative prices</h5>

In [76]:
v_tree = european_call.derivative_tree(s_tree)
#v_tree.T will transpose the tree so it looks like how we build them by hand
vt_df = pd.DataFrame(v_tree.T, columns=columns(european_call.n))
vt_df

,t = 0,t = 1,t = 2
0,1.282185,2.025584,3.2
1,0.000000,0.000000,0.0
2,0.000000,0.000000,0.0


<h5>Optimal price at t=0</h5>

In [77]:
european_call.price()

1.2821849452741405

<h3>Vanilla European Put Example</h3>

In [78]:
r = InterestRate(0.04, short_term=True)
european_put = EuropeanVanillaModel(80, 100, 2, 1.2, .8, r, 2, option='put')

<h5>Tree of stock prices</h5>

In [79]:
s_tree = european_put.stock_tree()
#s_tree.T will transpose the tree so it looks like how we build them by hand
st_df = pd.DataFrame(s_tree.T, columns=columns(european_put.n))
st_df

,t = 0,t = 1,t = 2
0,80.0,96.0,115.2
1,0.0,64.0,76.8
2,0.0,0.0,51.2


<h5>Tree of derivative prices</h5>

In [80]:
v_tree = european_put.derivative_tree(s_tree)
#v_tree.T will transpose the tree so it looks like how we build them by hand
vt_df = pd.DataFrame(v_tree.T, columns=columns(european_put.n))
vt_df

,t = 0,t = 1,t = 2
0,17.397114,8.870945,0.0
1,0.000000,32.078944,23.2
2,0.000000,0.000000,48.8


<h5>Optimal price at t=0</h5>

In [81]:
european_put.price()

17.397113747817905

<h3>American Put Example</h3>

In [82]:
r = InterestRate(0.04, short_term=True)
american_put = AmericanModel(80, 100, 2, 1.2, .8, r, 2, option='put')

<h5>Tree of stock prices</h5>

In [83]:
s_tree = american_put.stock_tree()
st_df = pd.DataFrame(s_tree, columns=columns(american_put.n))
st_df

,t = 0,t = 1,t = 2
0,80.0,0.0,0.0
1,96.0,64.0,0.0
2,115.2,76.8,51.2


<h5>Tree of derivative prices</h5>

In [84]:
v_tree = american_put.derivative_tree(s_tree.data)
vt_df = pd.DataFrame(v_tree, columns=columns(american_put.n))
vt_df

,t = 0,t = 1,t = 2
0,20.000000,0.0,0.0
1,8.870945,36.0,0.0
2,0.000000,23.2,48.8


<h5>Optimal price at t=0</h5>

In [85]:
american_put.price()

20.0